# 4.4. Model Selection, Underfitting, and Overfitting


## 4.4.4. Polynomial Regression


In [2]:
import math
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

In [6]:
max_degree = 20
n_train, n_test = 100, 100  
true_w = np.zeros(max_degree) 
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])

features = np.random.normal(size=(n_train + n_test, 1))
np.random.shuffle(features)
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1))
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1)  
labels = np.dot(poly_features, true_w)
labels += np.random.normal(scale=0.1, size=labels.shape)

In [7]:
# Convert from NumPy ndarrays to tensors
true_w, features, poly_features, labels = [torch.tensor(x, dtype=
    torch.float32) for x in [true_w, features, poly_features, labels]]

features[:2], poly_features[:2, :], labels[:2]

(tensor([[0.5791],
         [0.5143]]),
 tensor([[1.0000e+00, 5.7905e-01, 1.6765e-01, 3.2360e-02, 4.6845e-03, 5.4251e-04,
          5.2357e-05, 4.3311e-06, 3.1349e-07, 2.0170e-08, 1.1679e-09, 6.1482e-11,
          2.9668e-12, 1.3215e-13, 5.4658e-15, 2.1100e-16, 7.6362e-18, 2.6010e-19,
          8.3674e-21, 2.5501e-22],
         [1.0000e+00, 5.1426e-01, 1.3223e-01, 2.2667e-02, 2.9142e-03, 2.9973e-04,
          2.5690e-05, 1.8874e-06, 1.2132e-07, 6.9325e-09, 3.5651e-10, 1.6667e-11,
          7.1427e-13, 2.8256e-14, 1.0379e-15, 3.5584e-17, 1.1437e-18, 3.4598e-20,
          9.8847e-22, 2.6754e-23]]),
 tensor([5.3413, 5.5354]))

## 4.4.4.2. Training and Testing the Model


In [8]:
def evaluate_loss(net, data_iter, loss):
  metric = d2l.Accumulator(2)
  for X,y in data_iter:
    out = net(X)
    y = y.reshape(out.shape)
    metric.add(l.sum(), l.numel())
  return metric[0]/ metric[1]

In [10]:
def train(train_features, test_features, train_lables, test_labels, 
          num_epochs=400):
  loss = nn.MSELoss(reduction="none")
  input_shape = train_features.shape[-1]
  net = nn.Sequential(nn.Linear(input_shape, 1, bias=False))
  batch_size = min(10, train_lables.shape[0])
  train_iter = d2l.load_array((train_features, train_lables.reshape(-1,1)),batch_size)
  test_iter = d2l.load_array((test_features, test_labels.reshape(-1,1)), batch_size, is_train=False)
  trainer = torch.optim.SGD(net.parameters(), lr=0.01)
  animator = d2l.Animator(xlabel='epoch', ylabel='loss', yscale='log',
                            xlim=[1, num_epochs], ylim=[1e-3, 1e2],
                            legend=['train', 'test'])
  for epoch in range(num_epochs):
    d2l.train_epoch_ch3(net, train_iter, loss, trainer)
    if epoch == 0 or (epoch+1)%20 ==0:
      animator.add(epoch + 1, (evaluate_loss(net, train_iter, loss),
                                     evaluate_loss(net, test_iter, loss)))
  print('weight:', net[0].weight.data.numpy())

## 4.4.4.3. Third-Order Polynomial Function Fitting (Normal)


In [ ]:
train(poly_features[:n_train, :4], poly_features[n_train:, :4],
      labels[:n_train], labels[n_train:])


## 4.4.4.4. Linear Function Fitting (Underfitting)


In [ ]:
# Pick the first two dimensions, i.e., 1, x, from the polynomial features
train(poly_features[:n_train, :2], poly_features[n_train:, :2],
      labels[:n_train], labels[n_train:])

## 4.4.4.5. Higher-Order Polynomial Function Fitting (Overfitting)


In [ ]:
# Pick the first two dimensions, i.e., 1, x, from the polynomial features
train(poly_features[:n_train, :2], poly_features[n_train:, :2],
      labels[:n_train], labels[n_train:])